# Change Nyoka

In [ ]:
import inspect
#import PMML43Ext as pml
import sys
import nyoka.PMML43Ext as pml

In [ ]:
# Chunk 1
config1 = {'robo1': 25, 'robo2': 26}

def script1():
    r3 = r1+r2
    
chunk1 = {"config": config1, 'for_preprocessing': script1, 'for_scoring': script1}

# Chunk 2
config2 = {'robo1': 6666, 'robo2': 5555}

def script2():
    r6 = r1+r2+r3-r4
    
chunk2 = {"config": config2, 'for_preprocessing': script2}

In [ ]:
def skl_to_pmml(*arg):
    ll = len(arg)
    src = []
    for leng in range(ll):
        p1 = pml.script(content=arg[leng]['config'], for_="Config"+str(leng))
        p1.anyAttributes_ = {"type": "application/x-python"}
        p1.export(sys.stdout,1)
        p2 = pml.script(content=inspect.getsource(arg[leng]['for_preprocessing']), for_="for_preprocessing")
        p2.anyAttributes_ = {"model": "LGBM"}
        p2.export(sys.stdout,1)
        p3 = pml.script(content=inspect.getsource(arg[leng]['for_preprocessing']), for_="for_scoring")
        p3.anyAttributes_ = {"model": "XGB"}
        p3.export(sys.stdout,1)

In [ ]:
skl_to_pmml(chunk1,chunk2)

# Nyoka with Multiple Models - Installed

In [1]:
import nyoka
from nyoka import model_to_pmml

In [2]:
import pandas as pd
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn_pandas import DataFrameMapper
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

iris = datasets.load_iris()
irisd = pd.DataFrame(iris.data, columns=iris.feature_names)
irisd['Species'] = iris.target

features = irisd.columns.drop('Species')
target = 'Species'

pipeline_obj1 = Pipeline([
    ("mapping", DataFrameMapper([
    (['sepal length (cm)', 'sepal width (cm)'], StandardScaler()) , 
    (['petal length (cm)', 'petal width (cm)'], Imputer())
    ]))
])

pipeline_obj1.fit(irisd[features])
# pipeline_obj1.transform(irisd[features])

Pipeline(memory=None,
     steps=[('mapping', DataFrameMapper(default=False, df_out=False,
        features=[(['sepal length (cm)', 'sepal width (cm)'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['petal length (cm)', 'petal width (cm)'], Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0))],
        input_df=False, sparse=False))])

In [3]:
rff = RandomForestClassifier()
rff.fit(irisd[features], irisd[target])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [4]:
toExportDict={
    'model1':{
        'hyperparameters':None,
        'preProcessingScript':None,
        'pipelineObj':None,
        'modelObj':rff,
        'featuresUsed':features,
        'targetName':target,
        'postProcessingScript':None,
        'taskType': 'trainAndscore'
    }
}

pmml = model_to_pmml(toExportDict, pmml_f_name="chirag.pmml")

In [ ]:
import pandas as pd
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn_pandas import DataFrameMapper
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

iris = datasets.load_iris()
irisd = pd.DataFrame(iris.data, columns=iris.feature_names)
irisd['Species'] = iris.target

features = irisd.columns.drop('Species')
target = 'Species'

pipeline_obj2 = Pipeline([
    ("mapping", DataFrameMapper([
    (['sepal length (cm)', 'sepal width (cm)'], StandardScaler()) , 
    (['petal length (cm)', 'petal width (cm)'], Imputer())
    ])),
    ("rfc", RandomForestClassifier(n_estimators = 100))
])

pipeline_obj2.fit(irisd[features], irisd[target])

In [ ]:
toExportDict={
    'model1':{
        'hyperParameters':'',
        'preProcessingScript':{'scripts':[script1,script2], 'scriptpurpose':['train','score']},
        'pipelineObj':pipeline_obj1,
        'postProcessingScript':{'scripts':[script1], 'scriptpurpose':['postprocess']},
        'taskType': 'trainAndscore'
    },
    'model2':{
        'hyperParameters':'',
        'preProcessingScript':{'scripts':[script1,script2], 'scriptpurpose':['train','score']},
        'pipelineObj':pipeline_obj2,
        'postProcessingScript':{'scripts':[script1], 'scriptpurpose':['postprocess']},
        'taskType': 'score'
    }   
}

In [ ]:
skl_to_pmml(toExportDict, features, target, "multiple_models_pmml.pmml")

### Multiple MiningBuldingTask in Extensions

In [ ]:
def get_entire_string(pipe0):
    pipe_steps = pipe0.steps
    pipe_memory = 'memory=' + str(pipe0.memory)
    df_container = ''
    pipe_container = ''
    for step_idx, step in enumerate(pipe_steps):
        pipe_step_container = ''
        step_name = step[0]
        step_item = step[1]
        if step_item.__class__.__name__ == "DataFrameMapper":
            df_default_val = "default=" + str(step_item.default)
            df_out_val = "df_out=" + str(step_item.df_out)
            input_df_val = "input_df=" + str(step_item.input_df)
            sparse_val = "sparse=" + str(step_item.sparse)
            for feature in step_item.features:
                if not df_container:
                    df_container = df_container + str(feature)
                else:
                    df_container = df_container + ',' + str(feature)
            df_container = '[' + df_container + ']'
            df_container = 'features=' + df_container
            df_container = df_default_val + ',' + df_out_val + ',\n\t' + df_container
            df_container = df_container + ',\n\t' + input_df_val + ',' + sparse_val
            df_container = '(' + df_container + ')'
            df_container = 'DataFrameMapper' + df_container
            df_container = '\'' + step_name + '\'' + ',' + df_container
            df_container = '(' + df_container + ')'
        else:
            pipe_step_container = '\'' + step_name + '\'' + ',' + str(step_item)
            pipe_step_container = '(' + pipe_step_container + ')'
            if not pipe_container:
                pipe_container = pipe_container + pipe_step_container
            else:
                pipe_container = pipe_container + ',' + pipe_step_container
    if df_container:
        pipe_container = df_container + ',' + pipe_container
    pipe_container = '[' + pipe_container + ']'
    pipe_container = 'steps=' + pipe_container
    pipe_container = pipe_memory + ',\n    ' + pipe_container
    pipe_container = 'Pipeline(' + pipe_container + ')'

    return pipe_container

In [ ]:
def get_mining_buildtask(toExportDict):
    extension = []
    for model_name in toExportDict.keys():
        pipeline = toExportDict[model_name]['pipelineObj']
        pipeline = get_entire_string(pipeline)
        extension.append(pml.Extension(value=pipeline,name=model_name))
    mining_bld_task = pml.MiningBuildTask(Extension = extension)
    return mining_bld_task

In [ ]:
pmml = pml.PMML(
    #version=get_version(),
    #Header=get_header(),
    MiningBuildTask=get_mining_buildtask(toExportDict),
    #DataDictionary=get_data_dictionary(model, col_names, target_name, categoric_values),
    #**trfm_dict_kwargs,
    #**models_dict
)
pmml.export(sys.stdout,1)

### Script tags implementation

In [ ]:
toExportDict={
    'model1':{
        'hyperParameters':'',
        'preProcessingScript':{'scripts':[script1,script2], 'scriptpurpose':['train','score']},
        'pipelineObj':pipeline_obj1,
        'postProcessingScript':{'scripts':[script1], 'scriptpurpose':['postprocess']},
    },
    'model2':{
        'hyperParameters':'',
        'preProcessingScript':{'scripts':[script1,script2], 'scriptpurpose':['train','score']},
        'pipelineObj':pipeline_obj2,
        'postProcessingScript':{'scripts':[script1], 'scriptpurpose':['postprocess']},
    }   
}

In [ ]:
scrps = []
for model_name in toExportDict.keys():
    if toExportDict[model_name]['preProcessingScript'] is not None:
        lstlen = len(toExportDict[model_name]['preProcessingScript']['scripts'])
        for leng in range(lstlen):
            scrps.append(pml.script(content=inspect.getsource(toExportDict[model_name]['preProcessingScript']['scripts'][leng]), 
                                    for_= model_name, 
                                    class_ = 'preprocessing',
                                    scriptPurpose = toExportDict[model_name]['preProcessingScript']['scriptpurpose'][leng]
                                    ))
    if toExportDict[model_name]['postProcessingScript'] is not None:
        lstlen = len(toExportDict[model_name]['postProcessingScript']['scripts'])
        for leng in range(0,lstlen):
            scrps.append(pml.script(content=inspect.getsource(toExportDict[model_name]['postProcessingScript']['scripts'][leng]), 
                                    for_= model_name, 
                                    class_ = 'postprocessing',
                                    scriptPurpose = toExportDict[model_name]['postProcessingScript']['scriptpurpose'][leng]
                                   ))
        
pmml = pml.PMML(
    #version=get_version(),
    #Header=get_header(),
    #MiningBuildTask=get_mining_buildtask(toExportDict),
    #DataDictionary=get_data_dictionary(model, col_names, target_name, categoric_values),
    script = scrps,
    #**trfm_dict_kwargs,
    #**models_dict
)
pmml.export(sys.stdout,1)

### Questions

1) taskType only for RegressionModel?

2) Where will we have the information of what script will be used for pre-processings and which one for post-processing in PMML?

3) How should we pass the information of combining 3 model predictions (by mean etc.) in the PMML?

4) What if we have scenario like this (2 sklearn models for training, one Keras model for scoring). We have sklearn and Keras exporters separate :(

5) scriptPurpose - is it already supported?